In [ ]:
from PIL import Image
import os

output_path = 'output_jpg'
if not os.path.exists(output_path):
    os.makedirs(output_path)
# 打开大图像
with Image.open('2_17.jpg') as img:
    # 计算水平方向和垂直方向上可以切割出多少个256x256的图像
    width, height = img.size
    num_patches_x = width // 256
    num_patches_y = height // 256

    # 遍历所有可能的图像块
    for i in range(num_patches_x):
        for j in range(num_patches_y):
            # 计算当前图像块的位置
            left = i * 256
            upper = j * 256
            right = left + 256
            lower = upper + 256

            # 切割图像块
            crop_patchs = img.crop((left, upper, right, lower))

            # 保存切割后的图像块
            patch.save(os.path.join(output_path,f'patch_{i}_{j}.jpg'))


In [2]:
import cv2
import numpy as np

img_path = '../test/2_17.jpg'
input_width, input_height = 1280, 1280
img = cv2.imread(img_path)
print(img.shape)
img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img=cv2.resize(img, (input_width, input_height))
image_data = np.array(img) / 255.0
image_data = np.transpose(image_data, (2, 0, 1))
print(image_data.shape)

(2816, 4096, 3)
(3, 1280, 1280)
float64


In [ ]:
from extract_features_FM import Whole_Slide_Patchs
from torch.utils.data import DataLoader

wsi_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/xCY20010730-HSIL'
preprocess = None
target_patch_size = (224,224)
fine_grained = True
fine_grained_pre_size = (1280,1280)
fine_grained_size = (256,256)
dataset = Whole_Slide_Patchs(wsi_path=wsi_path, target_patch_size=target_patch_size, preprocess=preprocess,\
    fine_grained=fine_grained, fine_grained_pre_size=fine_grained_pre_size, fine_grained_size=fine_grained_size)
# dataset[0].shape
loader = DataLoader(dataset, batch_size=16, num_workers=20)
for data in loader:
    print(data.shape)

In [1]:
import onnxruntime as ort
import torch
import numpy as np

device_id=(1,2)
# 判断是使用GPU或CPU
providers = [
    [('CUDAExecutionProvider', {'device_id': id, }) for id in device_id]
#     'CPUExecutionProvider',  # 也可以设置CPU作为备选
]
model_path = '/home/huangjialong/projects/tctcls-lp/det-ljx/best_x7_20240822.onnx'
# 使用ONNX模型文件创建一个推理会话，并指定执行提供者
session = ort.InferenceSession(model_path, providers=providers)
model_inputs = session.get_inputs()

# 获取输入的形状，用于后续使用
# input_shape = model_inputs[0].shape
# input_name = model_inputs[0].name

# img = np.random.randn(10, 3, 1280, 1280).astype(np.float32)
# outputs = session.run(None, {input_name: img})

EP Error using [[('CUDAExecutionProvider', {'device_id': 1}), ('CUDAExecutionProvider', {'device_id': 2})]]
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.


RuntimeError: /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:124 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:117 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] CUDA failure 2: out of memory ; GPU=0 ; hostname=ubuntu ; expr=cudaDeviceSynchronize(); 



In [3]:
device_id=(1,2)

providers = [
    [('CUDAExecutionProvider', {'device_id': id, }) for id in device_id]
#     'CPUExecutionProvider',  # 也可以设置CPU作为备选
]
providers

[[('CUDAExecutionProvider', {'device_id': 1}),
  ('CUDAExecutionProvider', {'device_id': 2})]]

In [36]:
from extract_features_rtdetr import Whole_Slide_Patchs
from torch.utils.data import DataLoader
wsi_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/xCY20010730-HSIL'
target_patch_size = (1280,1280)
dataset = Whole_Slide_Patchs(wsi_path=wsi_path, target_patch_size=target_patch_size)
# dataset[0].shape
loader = DataLoader(dataset, batch_size=5, num_workers=20)

In [37]:

imgs = np.stack([dataset[120+i] for i in range(5)])
print(imgs.shape)
input_name = model_inputs[0].name
outputs = session.run(None, {input_name: img})


(5, 3, 1280, 1280)


In [32]:
from extract_features_rtdetr import postprocess_feats

    
confidence_thres=0.1
output_process = postprocess_feats(confidence_thres, outputs)
for output in output_process:
    print(type(output), output.shape)
# wsi_feat = []
# for data in DataLoader:
#     outputs = session.run(None, {input_name: data})
    

<class 'numpy.ndarray'> (2, 256)
<class 'numpy.ndarray'> (4, 256)
<class 'numpy.ndarray'> (5, 256)
<class 'numpy.ndarray'> (3, 256)
<class 'numpy.ndarray'> (3, 256)
<class 'numpy.ndarray'> (3, 256)
<class 'numpy.ndarray'> (4, 256)
<class 'numpy.ndarray'> (2, 256)
<class 'numpy.ndarray'> (3, 256)
<class 'numpy.ndarray'> (3, 256)
